## Imports and Global Functions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from datetime import datetime
from imports.project import Project

In [3]:
from fpdf import FPDF

## Setup Projects and Students

In [4]:
projects = [
    {
        'name': 'Metacognitive Reflection',
        'students': ['Victoria'],
        'sig': 'Agile Research Studios',
        'sig_abbr': 'ARS',
        'soap_url': 'https://docs.google.com/spreadsheets/d/1w7dbH1451XGOp1F2zFkrLg09Q0zDTPjhS4F2Bsl9OzY/edit?usp=sharing',
        'sheet_num': 1
    },
    {
        'name': 'The Weekly',
        'students': ['Shankar'],
        'sig': 'Agile Research Studios',
        'sig_abbr': 'ARS',
        'soap_url': 'https://docs.google.com/spreadsheets/d/1w7dbH1451XGOp1F2zFkrLg09Q0zDTPjhS4F2Bsl9OzY/edit?usp=sharing',
        'sheet_num': 2
    },
    {
        'name': 'Orchestration Scripts',
        'students': ['Richard'],
        'sig': 'Breaking Boundaries',
        'sig_abbr': 'BB',
        'soap_url': 'https://docs.google.com/spreadsheets/d/1w7dbH1451XGOp1F2zFkrLg09Q0zDTPjhS4F2Bsl9OzY/edit?usp=sharing',
        'sheet_num': 3
    },
    {
        'name': 'Regulation Devices',
        'students': ['Josh', 'Caryl'],
        'sig': 'Breaking Boundaries',
        'sig_abbr': 'BB',
        'soap_url': 'https://docs.google.com/spreadsheets/d/1w7dbH1451XGOp1F2zFkrLg09Q0zDTPjhS4F2Bsl9OzY/edit?usp=sharing',
        'sheet_num': 4
    },
    {
        'name': 'Scaffolded Exercises',
        'students': ['Maxine'],
        'sig': 'Readily Available Learning Experiences',
        'sig_abbr': 'RALE',
        'soap_url': 'https://docs.google.com/spreadsheets/d/1w7dbH1451XGOp1F2zFkrLg09Q0zDTPjhS4F2Bsl9OzY/edit?usp=sharing',
        'sheet_num': 5
    },
    {
        'name': 'Knowledge Maps',
        'students': ['Daniel'],
        'sig': 'Readily Available Learning Experiences',
        'sig_abbr': 'RALE',
        'soap_url': 'https://docs.google.com/spreadsheets/d/1w7dbH1451XGOp1F2zFkrLg09Q0zDTPjhS4F2Bsl9OzY/edit?usp=sharing',
        'sheet_num': 6
    },
    {
        'name': 'CE API',
        'students': ['Navin', 'Mary'],
        'sig': 'Collective Experiences',
        'sig_abbr': 'CE',
        'soap_url': 'https://docs.google.com/spreadsheets/d/1w7dbH1451XGOp1F2zFkrLg09Q0zDTPjhS4F2Bsl9OzY/edit?usp=sharing',
        'sheet_num': 7
    },
    {
        'name': 'Collective Narrative',
        'students': ['Sanfeng', 'Gabriel'],
        'sig': 'Collective Experiences',
        'sig_abbr': 'CE',
        'soap_url': 'https://docs.google.com/spreadsheets/d/1w7dbH1451XGOp1F2zFkrLg09Q0zDTPjhS4F2Bsl9OzY/edit?usp=sharing',
        'sheet_num': 8
    },
    {
        'name': 'Peer-to-Peer Delivery',
        'students': ['Cooper'],
        'sig': 'On-the-Go Crowdsourcing',
        'sig_abbr': 'OTG',
        'soap_url': 'https://docs.google.com/spreadsheets/d/1w7dbH1451XGOp1F2zFkrLg09Q0zDTPjhS4F2Bsl9OzY/edit?usp=sharing',
        'sheet_num': 9
    }
]

# Generate PDFs of Soap Notes

In [5]:
def normalize_string(x):
    return x.strip().encode('latin-1', 'ignore').decode('latin-1')

def add_soap_section(pdf, header_font, text_font, section_header, section_text):
    pdf.set_font(family='courier', style='U', size=header_font)
    pdf.cell(w=0, h=10, txt=section_header, ln=1)

    pdf.set_font(family='courier', size=text_font)
    pdf.multi_cell(w=0, h=5, txt=normalize_string(section_text))

In [6]:
# formatting for pdf
title_font_size = 18
soap_header_font_size = 14
soap_section_font_size = 12
text_font_size = 10

font_family = 'courier'

In [7]:
# generate SOAP pdfs for each project
for curr_proj in projects:
    # create a new Project for each project, and scrape SOAP notes for them
    proj = Project(curr_proj['name'], curr_proj['sig'], curr_proj['students'], curr_proj['soap_url'], curr_proj['sheet_num'])
    proj.scrape_soap_notes()
    
    # create pdf to add SOAP notes to
    pdf = FPDF()
    pdf.add_page()
    
    # add header for project name, sig, and students
    pdf.set_font(family=font_family, style='B', size=title_font_size)
    pdf.cell(w=0, h=10, txt='{} SOAP Notes'.format(proj.name), ln=1, align='C')
    pdf.cell(w=0, h=10, txt='Student(s): {}'.format(' and '.join(proj.students)), ln=2, align='C')
    pdf.cell(w=0, h=10, txt='SIG: {} ({})'.format(proj.sig, curr_proj['sig_abbr']), ln=2, align='C')
    pdf.ln()
    
    # add each SOAP note to pdf
    num_notes = len(proj.soap_notes)
    for index, curr_soap in enumerate(proj.soap_notes):
        # add header for soap note date
        curr_date = curr_soap.date.strftime('%m/%d/%Y')
        pdf.set_font(family=font_family, style='B', size=soap_header_font_size)
        pdf.cell(w=0, h=10, txt='SOAP Note from {} SIG'.format(curr_date), ln=1)

        # add each soap section
        add_soap_section(pdf, soap_section_font_size, text_font_size, 'Subjective or summary by the student', curr_soap.get_subjective())
        add_soap_section(pdf, soap_section_font_size, text_font_size, 'Objective data or information matching the student summary', curr_soap.get_objective())
        add_soap_section(pdf, soap_section_font_size, text_font_size, 'Assessment of the situation', curr_soap.get_assessment())
        add_soap_section(pdf, soap_section_font_size, text_font_size, 'Plan for follow-up', curr_soap.get_plan())
        
        # add page if still another soap note to add
        if index < num_notes - 1:
            pdf.add_page()
    
    # export pdf
    pdf.output('./documents/{}_{}_soap.pdf'.format(curr_proj['sig_abbr'].lower(),
                                                   proj.name.lower().replace(' ', '-')))